Installing packages to the right python version

In [ ]:
!/usr/local/opt/python/bin/python2.7 -m pip install Pillow

In [ ]:
!brew install openslide
!/usr/local/opt/python/bin/python2.7 -m pip install openslide-python

# This segment performs preprocessing and tiling

In [1]:
import glob
import os

from tqdm import tqdm

import config as cfg
import PIL
import cv2
from PIL import Image
from tiler import WholeSlideTiler


def main():
    # open input_path, and process each wholeslide image
    files = glob.glob(cfg.IMAGE_FOLDER_PATH + '/*.svs')
    for slidepath in tqdm(files):
        basename = os.path.splitext(os.path.basename(slidepath))[0]
        basepath = os.path.join(cfg.OUTPUT_FOLDER_PATH, basename)
        WholeSlideTiler(slidepath, basepath, cfg.IMG_FORMAT, cfg.TILE_SIZE, cfg.OVERLAP, cfg.LIMIT_BOUNDS, cfg.ROTATE,
                        cfg.QUALITY, cfg.NUM_WORKERS, cfg.ONLY_LAST).run()


if __name__ == '__main__':
    main()


Tiling slide: wrote 252/252 tilest/s]


Tiling completed on slide in:  <time here>


Tiling label: wrote 1/1 tiles
Tiling macro: wrote 2/2 tiles
Tiling thumbnail: wrote 1/1 tiles


Tiling completed on label in:  <time here>
Tiling completed on macro in:  <time here>
Tiling completed on thumbnail in:  <time here>


100%|██████████| 1/1 [00:49<00:00, 49.06s/it]


Installing packages for file manipulation

In [2]:
!pip install tqdm


h


# This segment selects 10 tiles, performs normalization and displays the 10 tiles

In [2]:
from __future__ import division

import glob

import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image 
from numpy import random
import os


def safe_ln(x, minval=0.0000000001):
    return np.log(x.clip(min=minval))


def normalize_staining(I=None):
    I = I.astype(np.float64)
    Io = 240
    beta = 0.15
    alpha = 1
    HERef = np.array([[0.5626, 0.2159], [0.7201, 0.8012], [0.4062, 0.5581]])
    maxCRef = np.array([1.9705, 1.0308])

    (h, w, c) = np.shape(I)
    I = np.reshape(I, (h * w, c), order='F')

    # Step 1. Convert RGB to OD.
    OD = - np.log((I + 1) / Io) # optical density where each channel in the image is normalized to values between [0, 1]

    # Step 2. Remove data with OD intensity less than beta
    ODhat = (OD[(np.logical_not((OD < beta).any(axis=1))), :])

    # Step 3. Calculate SVD on the OD tuples
    cov = np.cov(ODhat, rowvar=False)
    (W, V) = np.linalg.eig(cov)

    # Step 4. create plane from the SVD directions
    # corresponding to the two largest singular values
    Vec = - np.transpose(np.array([V[:, 1], V[:, 0]]))

    # Step 5. Project data onto the plane and normalize to unit Length
    That = np.dot(ODhat, Vec)

    # Step 6. Calculate angle of each point w.r.t the first SVD direction
    phi = np.arctan2(That[:, 1], That[:, 0])

    # Step 7. Find robust extremes (some alpha th and (100 - alpha th) percentiles of the angle
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)
    vMin = np.dot(Vec, np.array([np.cos(minPhi), np.sin(minPhi)]))
    vMax = np.dot(Vec, np.array([np.cos(maxPhi), np.sin(maxPhi)]))
    if vMin[0] > vMax[0]:
        HE = np.array([vMin, vMax])
    else:
        HE = np.array([vMax, vMin])

    HE = np.transpose(HE)

    # Step 8. Convert extreme values back to OD space
    Y = np.transpose(np.reshape(OD, (h * w, c)))
    C = np.linalg.lstsq(HE, Y,rcond=None)
    maxC = np.percentile(C[0], 99, axis=1)
    C = C[0] / maxC[:, None]
    C = C * maxCRef[:, None]
    Inorm = Io * np.exp(- np.dot(HERef, C))
    Inorm = np.reshape(np.transpose(Inorm), (h, w, c), order='F')
    Inorm = np.clip(Inorm, 0, 255)
    Inorm = np.array(Inorm, dtype=np.uint8)

    return Inorm  # ,H,E


def main():

    list_img = glob.glob(glob.glob("output/*/")[0]+"slide/15/*.jpeg")
    for filename in random.choice(list_img,10):
        fileout = os.path.splitext(filename)[0]+"_normal.jpeg"
        img = cv2.imread(filename)
        img = img.astype(np.float64)

        img = normalize_staining(img)

        cv2.imwrite(fileout, img)
        img_show = Image.open(fileout)
        img_show.show()  


if __name__ == '__main__':
    main()
